# Combining Python, SQL DBs and AWS

## Aim: Using python, create a DB on your AWS server, insert data into it and query that data to answer questions. 


# Using Python with MYSQL DB

In [1]:
# make sure we have the package installed
!pip install mysql-connector-python

In [2]:
## importing 'mysql.connector' 
import mysql.connector 

In [3]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "dbms"
)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost' (61)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [ ]:
import config

In [ ]:
config.pw_example

In [ ]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = "database-1.cl7l8mgndgqj.us-east-2.rds.amazonaws.com",
    user = "KuSin",
    passwd = "OneP!ece12"
)

print(cnx) # it will print a connection object if everything is fine


In [ ]:
cursor = cnx.cursor()

## Creating DB

In [4]:
import mysql.connector
from mysql.connector import errorcode

In [5]:
db_name = 'students'

Function to help us create a database

In [6]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [7]:
# check to see if the database already 
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

NameError: name 'cursor' is not defined

### Creating tables

In [8]:
DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [9]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [10]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

NameError: name 'config' is not defined

In [11]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: 

NameError: name 'cursor' is not defined

## Inserting Data

In [12]:
from datetime import date, datetime, timedelta

#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor()

NameError: name 'config' is not defined

In [13]:
tomorrow = datetime.now().date() + timedelta(days=1)

In [14]:
# insert statement to add an employee
add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")

#insert statement to add a salary
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

#data to insert for one employee
data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [ ]:
# Insert new employee
cursor.execute(add_employee, data_employee)
# Make sure data is committed to the database
cnx.commit()

In [ ]:
# grabs the id of the employee we just inserted so we can use it to input salary data
emp_no = cursor.lastrowid

In [ ]:

# create salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}

In [ ]:

#Insert that salary information
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

## Insert Many

If you have a list of records that you would like to input, there are two ways you can do this.  YOu can loop over each record in python and execute each record, or you can use the `executemany()` method.  

`cursor.executemany(operation, list_of_params)`



In [ ]:
#create a list of records

data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]

In [ ]:
#create the insert statment
stmt = "INSERT INTO  (first_name, hire_date) VALUES (%s, %s)"


In [ ]:
#insert all of the records and commit it
cursor.executemany(stmt, data)
cnx.commit()

In [ ]:
#Close the connection 

cursor.close()
cnx.close()

## Quyerying the DB

In [ ]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [ ]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime.date(2006, 1, 1)
hire_end = datetime.date(2006, 12, 31)

cursor.execute(query, (hire_start, hire_end))



### How to read the data from your query.

You can use the `fetchone()`, `fetchmany()`, or `fetchall()` methods.
 


The `fetchone()` method returns the next row of a query result set or `None` in case there is no row left.

        row = cursor.fetchone()
 
        while row is not None:
            print(row)
            row = cursor.fetchone()

In case the number of rows in the table is small, you can use the `fetchall()` method to fetch all rows from the database table.

        rows = cursor.fetchall()
 
        print('Total Row(s):', cursor.rowcount)
        for row in rows:
            print(row)

**Why might you use a fetchone() when you have a query that will return multiple rows?**

For a relatively big table, it takes time to fetch all rows and return the entire result set. In addition, `fetchall()` needs to allocate enough memory to store the entire result set in the memory, which is not efficient.

MySQL Connector/Python has the `fetchmany()` method that returns the next number of rows (n) of the result set, which allows you to balance between retrieval time and memory space.



First, develop a generator that chunks the database calls into a series of `fetchmany()` calls:

In [ ]:
def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

Second, use the  iter_row() generator to fetch 10 rows at a time :

`cursor.execute("SELECT * FROM books")`
 
`for row in iter_row(cursor, 10):
    print(row)`

In [ ]:
#in practice

data = cursor.fetchall()

In [ ]:
print(data)

In [ ]:
cursor.close()
cnx.close()

## Applied: Working with our student data

### Import our student data

In [ ]:
import json

f=open('students.json','r')
data=json.load(f)

In [ ]:
#examine that student data

### Determine how you should define your table to hold the data



### Create a connection and cursor to AWS db

In [ ]:
# 

### Create a table for our student info

In [ ]:

create_query = """

);
"""

### Now that we have created our table we need to insert our data into the table.

We can't just pass the execute query a list of dictioanries,  We need to parse through it first. 

Take one student and identify how you would parse that information into a usable format.

In [ ]:
test_student = data[0]

Now use a for loop to loop over all of data and return a list that we can use for `executemany()`.

**If all of your data isn't formatted as expected, this will cause your loop to fail**  you might have to account for 'bad data' in your for loop.

In [ ]:
students_list = []
    
for student in data:

    student_tuple= ()
    students_list.append(student_tuple)

Now that we have this list of tuples, lets use `executemany()` to insert the data.

Now that we have the data in the database, we can write

### Write queries to answer the following questions:

**Questions**
- Which student was born closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- Do any two students have the same favorite food?